# Project

Imagine that you have a huge dataset. And you want to quickly look at it/analyze it.

You already used pandas for this purpose. But you can't use pandas for huge data sets.

## Small dataset example

Dataset can be considered as a key-value storage

In [1]:
import pandas as pd
df = pd.read_csv('../data/yerevan_april_9.csv', sep=',')
df.head()

,idx,price,condition,district,max_floor,street,num_rooms,region,area,url,num_bathrooms,building_type,floor,ceiling_height
0,0,65000.0,good,Center,9,Vardanants St,3,Yerevan,80.0,http://www.myrealty.am/en/item/24032/3-senyaka...,1,panel,4,2.8
1,1,140000.0,newly repaired,Arabkir,10,Hr.Kochar St,4,Yerevan,115.0,http://www.myrealty.am/en/item/17599/4-senyaka...,1,monolit,2,3.0
2,2,97000.0,newly repaired,Center,10,Teryan St,2,Yerevan,72.0,http://www.myrealty.am/en/item/29472/2-senyaka...,1,panel,3,2.8
3,3,47000.0,good,Center,9,D. Demirchyan St,1,Yerevan,43.0,http://www.myrealty.am/en/item/37186/1-senyaka...,1,panel,9,2.8
4,4,51000.0,newly repaired,Center,14,Sayat Nova Ave,1,Yerevan,33.0,http://www.myrealty.am/en/item/37184/1-senyaka...,1,other,4,2.8


Above idx is a key and it's uniq among the rows.
Let's calculate average, min and max prices of appartment in each district.
That means, we don't care about columns other than district and price.

In [2]:
df = df[['price', 'district','area']]
df['price_per_meter'] = df.price / df.area
df[0:5]

,price,district,area,price_per_meter
0,65000.0,Center,80.0,812.500000
1,140000.0,Arabkir,115.0,1217.391304
2,97000.0,Center,72.0,1347.222222
3,47000.0,Center,43.0,1093.023256
4,51000.0,Center,33.0,1545.454545


In [3]:
grouped = df.groupby(['district'])['price_per_meter']
list(grouped)[0:2]

[('Achapnyak', 19       494.505495
  25       620.689655
  27       347.560976
  28       636.363636
  43       403.846154
  50       437.500000
  53       745.454545
  61       837.837838
  97       416.666667
  106      428.571429
  114      451.612903
  116      750.000000
  124      527.272727
  131      828.571429
  137      738.095238
  150      625.000000
  151     1066.666667
  224      619.047619
  226      958.333333
  232      868.421053
  243      670.731707
  244      617.977528
  351      787.037037
  386      723.076923
  421      593.750000
  481      507.692308
  486      423.076923
  490      666.666667
  491      395.061728
  494      432.432432
             ...     
  5299     789.473684
  5300     660.000000
  5301     635.593220
  5302     580.000000
  5304     471.153846
  5306     837.837838
  5396     560.747664
  5678     854.545455
  5706     727.272727
  5727     661.971831
  5731     687.500000
  5788     813.953488
  5958     764.044944
  5961     716.4179

In [4]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}
answer = grouped.apply(get_stats).unstack()
answer

,count,max,mean,min
district,,,,
Achapnyak,246.0,1111.111111,656.650121,347.560976
Arabkir,1640.0,2666.666667,991.604966,335.714286
Avan,320.0,1750.000000,734.465192,367.021277
Center,2588.0,3928.571429,1361.005018,400.000000
Davtashen,309.0,1950.000000,801.170345,364.485981
Erebuni,94.0,939.024390,622.417553,333.333333
Malatia-Sebastia,292.0,1000.000000,580.504672,257.142857
Nor Norq,291.0,1087.500000,617.894857,315.315315
Norq Marash,5.0,1491.666667,1286.641853,769.230769


## Let's try on a bigger dataset

In [5]:
%time df = pd.read_csv('../data/1000000 Sales Records.csv', sep=',', low_memory=False)

CPU times: user 2.25 s, sys: 332 ms, total: 2.58 s
Wall time: 2.11 s


It took 2s and 500Mb of memory just to load 125Mb dataset. Usually, it's necessary to work with up to 100Tb datasets.

## Mapreduce

A way to process huge datasets without loading them to memory.

A way to preprocess each data row separately (map stage). This may be done in parallel.

And then - join all the rows with post-processing (reduce stage).

A good article about mapreduce https://www.tutorialspoint.com/hadoop/hadoop_mapreduce.htm
![Mapreduce](https://www.tutorialspoint.com/hadoop/images/mapreduce_algorithm.jpg)

But mapreduce needs an infrastructure. YT at Yandex, BigTable at Google, Hadoop + Spark - opensource solution used by almost everybody else.

## Csvtools

It's a set of toy mapreduce utils aimed to be used on a local machine from command line.

You are supposed to construct a dataprocessing pipeline using csvtools redirecting output of one command to another command's input using pipes ('|').

You'll need either linux or mac terminal or (cygwin+python or virtual box with ubuntu or docker) on windows to utilize pipes. Probably, powershell will be enough but I am not sure.

### csvpp, csvcut

csvpp is just a pretty print.

csvcut is a way to preserve only the desired columns

In [1]:
!head -10 yerevan_april_9.csv | csvcut -f price,district,area | csvpp -f

price    | district  | area
---------------------------
65000    | Center    | 80.0
1.4e+05  | Arabkir   | 115 
97000    | Center    | 72.0
47000    | Center    | 43.0
51000    | Center    | 33.0
44900    | Center    | 71.0
67000    | Arabkir   | 60.0
1.43e+05 | Center    | 98.0
27000    | Shengavit | 48.0


The idea is to distribute your dataset over multiple machines

### csvmap

csvmap is a way to define a new column or modify an existing one. It applies the transformation row-wide. As the map step in mapreduce.


In [1]:
!head -10 yerevan_april_9.csv | csvcut -f price,district,area | csvmap '$price_per_meter = $price / $area' | csvpp -f

price    | district  | area | price_per_meter
---------------------------------------------
65000    | Center    | 80.0 | 812.5          
1.4e+05  | Arabkir   | 115  | 1217.4         
97000    | Center    | 72.0 | 1347.2         
47000    | Center    | 43.0 | 1093           
51000    | Center    | 33.0 | 1545.5         
44900    | Center    | 71.0 | 632.39         
67000    | Arabkir   | 60.0 | 1116.7         
1.43e+05 | Center    | 98.0 | 1459.2         
27000    | Shengavit | 48.0 | 562.5          


### csvreduce

csvreduce is a way to calculate column statistics per each key(s) value. In the following example it's used to calculate mean price_per_meter.

In [1]:
!cat yerevan_april_9.csv | csvcut -f price,district,area | csvmap '$price_per_meter = $price / $area' | csvreduce -k district -r avg[price_per_meter],count | csvpp -f

district         | avg_price_per_meter | count
----------------------------------------------
Achapnyak        | 656.65              | 246  
Arabkir          | 991.6               | 1640 
Avan             | 734.47              | 320  
Center           | 1361                | 2588 
Davtashen        | 801.17              | 309  
Erebuni          | 622.42              | 94   
Malatia-Sebastia | 580.5               | 292  
Nor Norq         | 617.89              | 291  
Norq Marash      | 1286.6              | 5    
Nubarashen       | 576.92              | 1    
Qanaqer-Zeytun   | 756.05              | 239  
Shengavit        | 642.44              | 206  
Vahagni district | 1071.4              | 3    


Woo-hoo! We analysed our dataset with a single line

### csvplot

And even plot the results

In [0]:
!cat yerevan_april_9.csv | csvcut -f price,district,area | csvmap '$price_per_meter = $price / $area' | csvreduce -k district -r avg[price_per_meter],count | csvplot -y avg_price_per_meter

![csvplot output](./csvplot_example.png)

### csvhead, csvtail

Same as head and tail but work correctly with comma-separated files.

## Your homework

Implement that useful set of tools:)

Formal specifications and some tips are available in my github repo https://github.com/aelphy/csvtools

# Description is here:

https://docs.google.com/document/d/1WLjxKzaQn599E2okQE_JCMYOUe77iaIHjBIvZ4XqOfY/edit?usp=sharing
    
https://docs.google.com/document/d/1DBtKKW_7QZbrQPPDy9b7YmwTwA44PsnKRolQktEd2fc/edit?usp=sharing